In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import time
from multiprocessing import Pool

In [2]:
df_no_time = pd.read_csv(Path(r"../data/blue_waters_posix_with_paths_no_negative_outliers_no_time.csv"))

In [3]:
columns_dissimilar = [
    "POSIX_MAX_READ_TIME_SIZE", 
    "POSIX_MAX_WRITE_TIME_SIZE", 
    "POSIX_FASTEST_RANK", 
    "POSIX_SLOWEST_RANK",
    "rank",
    "POSIX_TOTAL_TIME",
]

In [4]:
count = df_no_time.groupby(by=list(df_no_time.columns.difference(columns_dissimilar))).agg({"POSIX_TOTAL_TIME":['mean','count']})
count.head()

POSIX_TOTAL_TIME  \
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          mean   
POSIX_ACCESS1_ACCESS POSIX_ACCESS1_COUNT POSIX_ACCESS2_ACCESS POSIX_ACCESS2_COUNT POSIX_ACCESS3_ACCESS POSIX_ACCESS3_COUNT POSIX_ACCESS4_ACCESS POSIX_ACCESS4_COUNT POSIX_BYTES_READ POSIX_BYTES_WRITTEN POSIX_CONSEC_READS POSIX_CONSEC_WRITES POSIX_DUPS POSIX_FASTEST_RANK_BYTES POSIX_FILENOS POSIX_FILE_ALIGNMENT POSIX_FILE_NOT_ALIGNED POSIX_FSYNCS POSIX_MAX_BYTE_READ POSIX_MAX_BYTE_WRITTEN POSIX_MEM_ALIGNMENT POSIX_MEM_NOT_ALIGNED POSIX_MMAPS POSIX_MODE POSIX_OPENS POSIX_READS POSIX_RENAME_SOURCES POSIX_RENAME_TARGETS POSIX_RW_SWITCHES POSIX_SEEKS POSIX_SEQ_READS POSIX_SEQ_WRITES POSIX_SIZE_READ_0_100 POSIX_SIZE_READ_100K_1M POSIX_SIZE_READ_100M_1G POSIX_SIZE_READ_100_1K POSIX_SIZE_READ_10K_100K POSIX_SIZE_READ_10M_100M POSIX_SIZE_READ_1G_PLUS POSIX_SIZE_READ_1K_10K POSIX_SIZE_READ_1M_4M POSIX_SIZE_READ_4M_10M POSIX_SIZE_WRITE_0_100 POSIX_SIZE_WRITE_100K_1M POSIX_SIZE_WRITE_100M_1G POSIX_SIZE_WRITE_100_1K POSIX_SIZE_WRITE_10K_100K POSIX_SIZE_WRITE_10M_100M POSIX_SIZE_WRITE_1G_PLUS POSIX_SIZE_WRITE_1K_10K POSIX_SIZE_WRITE_1M_4M POSIX_SIZE_WRITE_4M_10M POSIX_SLOWEST_RANK_BYTES POSIX_STATS POSIX_STRIDE1_COUNT POSIX_STRIDE1_STRIDE POSIX_STRIDE2_COUNT POSIX_STRIDE2_STRIDE POSIX_STRIDE3_COUNT POSIX_STRIDE3_STRIDE POSIX_STRIDE4_COUNT POSIX_STRIDE4_STRIDE POSIX_WRITES READ_0_100 READ_100K_1M READ_100M_1G READ_100_1K READ_10K_100K READ_10M_100M READ_1G_PLUS READ_1K_10K READ_1M_4M READ_4M_10M WRITE_0_100 WRITE_100K_1M WRITE_100M_1G WRITE_100_1K WRITE_10K_100K WRITE_10M_100M WRITE_1G_PLUS WRITE_1K_10K WRITE_1M_4M WRITE_4M_10M exe        lustre nprocs path                                                                  
0                    0                   0                    0                   0                    0                   0                    0                   0                0                   0                  0                   -163148    0                        -163148       171073077248         0                      0            0                   0                      1305184             0                     0           0          0           0           -163148              -163148              0                 0           0               0                0                     0                       0                       0                      0                        0         

In [5]:
indices = df_no_time.groupby(by=list(df_no_time.columns.difference(columns_dissimilar))).indices 

In [6]:
df_no_time_cpy = df_no_time.copy().reset_index()
df_no_time_cpy.head()

,index,POSIX_OPENS,POSIX_FILENOS,POSIX_DUPS,POSIX_READS,POSIX_WRITES,POSIX_SEEKS,POSIX_STATS,POSIX_MMAPS,POSIX_FSYNCS,...,WRITE_4M_10M,WRITE_10M_100M,WRITE_100M_1G,WRITE_1G_PLUS,rank,POSIX_TOTAL_TIME,nprocs,exe,lustre,path
0,0,1280,-1281,-1281,1807,0,0,2561,0,0,...,0,0,0,0,184,0.970498,213,Hsigma,1,/hpcwork/noco0056/io_transfer_learning/data/bl...
1,1,168420,-5232,-5232,7505724,5401751,11345341,394840,0,1,...,0,0,0,0,0,147.540409,80,SpEC,1,/hpcwork/noco0056/io_transfer_learning/data/bl...
2,2,62567,-1477,-1477,2300215,441912,1214829,280660,0,1,...,0,0,0,0,-1,24.994056,48,SpEC,1,/hpcwork/noco0056/io_transfer_learning/data/bl...
3,3,2366,-2367,-2367,3342,0,0,4733,0,0,...,0,0,0,0,127,0.487127,394,Hsigma,1,/hpcwork/noco0056/io_transfer_learning/data/bl...
4,4,8709,-288,-288,1521073,6331589,2013343,28430,0,0,...,0,0,0,0,-1,14389.735087,256,CCTM_v52_Linux2_x86_64intel,1,/hpcwork/noco0056/io_transfer_learning/data/bl...


In [7]:
for a in indices:        
    idx = indices[a]
    dup_set = df_no_time_cpy.loc[idx]

    if len(dup_set) == 1:
        df_no_time_cpy.loc[idx, 'error'] = 0
        continue

    mean = dup_set['POSIX_TOTAL_TIME'].mean()
    df_no_time_cpy.loc[idx, 'mean'] = mean
    df_no_time_cpy.loc[idx, 'error'] = dup_set['POSIX_TOTAL_TIME'] - mean

    # count = len(dup_set)
    # bessel = count / (count - 1) if count > 1 else 1
    # df_no_time_cpy.loc[idx, 'error'] = (dup_set['POSIX_TOTAL_TIME'] - mean) * bessel

In [8]:
df_no_time_cpy = df_no_time_cpy.drop("index", axis=1)

In [9]:
df_no_time_cpy.to_csv(Path(r"../data/blue_waters_posix_with_paths_no_negative_outliers_no_time_witherrors.csv"),index=False)

In [10]:
df_no_time_cpy['error'].abs().median() / df_no_time_cpy.POSIX_TOTAL_TIME.mean()

0.0